In [1]:
# Cell 0
import os, sys, random
import numpy as np
import torch

print("CUDA:", torch.cuda.is_available())

CUDA: True


In [2]:
# Cell 1
from omegaconf import OmegaConf
from torch.nn import functional as F

from src.utils.dataset import get_dataset
from src.datasets.dataset import DataInfo
from src.utils.models import get_model

# If you want to reuse your training pipeline:
from src.oracles.train.train import Trainer